In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [8]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI')
genai.configure(api_key=GEMINI_API_KEY)

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.4,google_api_key=GEMINI_API_KEY)

In [10]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
        "When looking for information on internet use 'search_query' as input argument for search tools provided"
    ),
    allow_delegation=False,
    verbose=True,
    llm = llm
)

In [23]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
        "When looking for information on internet use 'search_query' as input argument for search tools provided"
    ),
    allow_delegation=False,
    verbose=True,
    llm = llm
)

In [24]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [25]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [26]:
search_tool._run(search_query="DeeplearningAI")

"\nSearch results: Title: DeepLearning.AI: Start or Advance Your Career in AI\nLink: https://www.deeplearning.ai/\nSnippet: DeepLearning.AI | Andrew Ng | Join over 7 million people learning how to use and build AI through our online courses. Earn certifications, level up your ...\n---\nTitle: DeepLearningAI - YouTube\nLink: https://www.youtube.com/c/deeplearningai\nSnippet: Welcome to the official DeepLearning.AI YouTube channel! Here you can find the videos from our Coursera programs on machine learning as well as recorded ...\n---\nTitle: DeepLearning.AI Online Courses - Coursera\nLink: https://www.coursera.org/deeplearning-ai\nSnippet: DeepLearning.AI's expert-led educational experiences provide AI practitioners and non-technical professionals with the necessary tools to go all the way from ...\n---\nTitle: DeepLearning.AI - LinkedIn\nLink: https://www.linkedin.com/company/deeplearningai\nSnippet: Online Course Platforms. Learn the skills to start or advance your AI career | World-c

In [27]:
from crewai_tools import BaseTool

In [28]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [29]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [30]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [31]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [32]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True,
    max_rpm=15,
    embedder={
            "provider": "google",
            "config":{
                "model": 'models/text-embedding-004',
                "task_type": "retrieval_document",
                "title": "Embeddings for Embedchain"
            }
    } 
)

2024-07-04 01:44:08,678 - 8764 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [33]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Action: Search the internet
Action Input: {"search_query": "DeepLearningAI"} 


Search results: Title: DeepLearning.AI: Start or Advance Your Career in AI
Link: https://www.deeplearning.ai/
Snippet: DeepLearning.AI | Andrew Ng | Join over 7 million people learning how to use and build AI through our online courses. Earn certifications, level up your ...
---
Title: DeepLearningAI - YouTube
Link: https://ww

In [34]:
from IPython.display import Markdown
Markdown(result)

Hi Andrew,

I hope this email finds you well.

I'm writing to you today to introduce DeepLearning.AI and our suite of online courses and specializations in AI. I believe that our offerings can help DeepLearning.AI achieve its goals of making world-class AI education accessible to individuals worldwide.

I'm particularly excited about our new course, "Building Systems with the ChatGPT API." This course teaches students how to use the ChatGPT API to build powerful AI-powered applications. I believe that this course would be a valuable addition to DeepLearning.AI's curriculum.

In addition to our courses, we also offer a variety of other services that can help DeepLearning.AI achieve its goals. These services include:

* Custom Training Programs: We can develop customized training programs tailored to the specific needs of your organization.
* Partnerships with Universities: We can partner with universities to offer joint degree programs or certifications in AI.
* AI Consulting Services: We can leverage our expertise to provide consulting services to businesses looking to implement AI solutions.

I would be happy to schedule a time to discuss our offerings in more detail. Please let me know if you have any questions.

Thanks,
[Your Name]